<a href="https://colab.research.google.com/github/deRockerTom/dataFrenchWords/blob/main/compressFrenchFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/deRockerTom/dataFrenchWords
!pip install textract
!sudo apt-get install antiword

Cloning into 'dataFrenchWords'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 63 (delta 24), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (63/63), done.


In [4]:
import textract
text = textract.process("dataFrenchWords/maupassant_bel_ami_illustre.doc")
text = text.decode("utf-8")
text = text.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '')

In [5]:
# Get dict with file
import os

def getDictWithFile(file):
  with open(file, 'r') as f:
    dic = {}
    key = ""
    value = ""
    isKey = True
    for l in f.readlines():
      for c in l:
        if c == 'ក':
          isKey = False
        elif c == 'ខ':
          isKey = True
          dic[key] = int(value)
          key = ""
          value = ""
        else:
          if isKey:
            key = key + c
          else:
            value = value + c
    value = value + "\n"
  return dic

dicByHugo = getDictWithFile('dataFrenchWords/dicByHugo')
dicByHugoMax = getDictWithFile('dataFrenchWords/dicByHugoMax')
dictByHugoMax10 = getDictWithFile('dataFrenchWords/dicByHugoMax10')
dictByHugoMax50 = getDictWithFile('dataFrenchWords/dicByHugoMax50')




In [6]:
def convertToBinary(number):
  binary = ''
  while number > 0:
    binary = str(number % 2) + binary
    number = number // 2
  return binary
def convertToNineBits(bits):
  n = len(bits)
  if n < 9:
    k = 9 - n
    return str('0') * k + bits
  elif n == 9:
    return bits
  else:
    print("Attention : un des encodages est trop grand (>9)")
    print(bits)

convertToNineBits(convertToBinary(255))

'011111111'

In [9]:
import csv

# open csv
dic = {}
with open('dataFrenchWords/french-word-list-verbs.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-adjectives.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

with open('dataFrenchWords/french-word-list-nouns.csv', 'r') as csvfile:
  reader = csv.reader(csvfile)
  #drop first 3 lines
  next(reader)
  next(reader)
  next(reader)
  #create dictionary
  for row in reader:
    if row[1] in dic:
      dic[row[1]] += int(row[2])
    else:
      dic[row[1]] = int(row[2])
  csvfile.close()

# Ponctuation :

dic[', '] = 150000
dic['. '] = 150000

dic = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
dic = {k: dic[k] for k in list(dic)[:256]} # In order to code the dictionnary on only 2 other bytes with most frequent words
offset = 256
i = 0
for key in dic:
  dic[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic2 = {}
offset = 256
for key in dicByHugo:
  dic2[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic3 = {}
offset = 256
for key in dicByHugoMax:
  dic3[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic4 = {}
offset = 256
for key in dictByHugoMax10:
  dic4[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1
dic5 = {}
offset = 256
for key in dictByHugoMax50:
  dic5[key] = convertToNineBits(convertToBinary(offset + i))
  offset += 1

print(dic5)

reverted_dic = {v: k for k, v in dic.items()}
reverted_dic2 = {v: k for k, v in dic2.items()}
reverted_dic3 = {v: k for k, v in dic3.items()}
reverted_dic4 = {v: k for k, v in dic4.items()}
reverted_dic5 = {v: k for k, v in dic5.items()}

print(reverted_dic5)

{'  ': '100000000', 'e ': '100000001', 's ': '100000010', 't ': '100000011', ' d': '100000100', ', ': '100000101', 'ai': '100000110', 'es': '100000111', ' l': '100001000', 'le': '100001001', 'en': '100001010', 're': '100001011', 'ou': '100001100', 'de': '100001101', ' s': '100001110', 'nt': '100001111', 'it': '100010000', ' p': '100010001', 'on': '100010010', ' c': '100010011', 'an': '100010100', 'n ': '100010101', ' e': '100010110', 'ur': '100010111', 'te': '100011000', 'er': '100011001', 'is': '100011010', 'r ': '100011011', '\n ': '100011100', ' a': '100011101', '\n\n': '100011110', 'a ': '100011111', 'qu': '100100000', '   ': '100100001', 'se': '100100010', 'ne': '100100011', 'et': '100100100', ' m': '100100101', 'me': '100100110', 'la': '100100111', ' de': '100101000', 'ie': '100101001', 'il': '100101010', 'e  ': '100101011', 'eu': '100101100', ' t': '100101101', 'ra': '100101110', 'ar': '100101111', 'in': '100110000', 'll': '100110001', 'ui': '100110010', 'l ': '100110011', 'es '

In [10]:
# class to build a tree defining all the possible  string in dic
class SuffixTree():
  class Node():
    def __init__(self,char):
      self.char = char
      self.out = {}
      self.isFinal = False
    def __str__(self, level=0):
      ret = "  " * level + str(self.char) + "\n"
      for c in self.out:
        ret += self.out[c].__str__(level + 1)
      return ret

  def __init__(self, dic):
    self.root = self.Node(None)
    self.root.isFinal = True
    for key in dic:
      cursor = self.root
      for c in key:
        if c in cursor.out:
          cursor = cursor.out[c]
        else:
          node = self.Node(c)
          cursor.out[c] = node
          cursor = node
      cursor.isFinal = True
  def longestPrefix(self, s):
    cursor = self.root
    word = ''
    temp = ''
    end = False
    i = 0
    length = len(s)
    while not(end) and i < length:
      if s[i] in cursor.out:
        temp = temp + s[i]
        cursor = cursor.out[s[i]]
        if cursor.isFinal:
          word = temp
        i += 1
      else:
        end = True
    return word
  def __str__(self):
    return f'{self.root}'

s = SuffixTree({'ab' : 1, 'abc': 2, 'coucou' : 5, 'aba' : 4, 'aaa' : 8})
print(s)
s.longestPrefix('abdc')
    


None
  a
    b
      c
      a
    a
      a
  c
    o
      u
        c
          o
            u



'ab'

In [14]:
tree = SuffixTree(dic)
tree2 = SuffixTree(dic2)
tree3 = SuffixTree(dic3)
tree4 = SuffixTree(dic4)
tree5 = SuffixTree(dic5)

In [12]:
def compress(string, tree, dic):
  i = 0
  ret = ''
  while i<len(string):
    temp = tree.longestPrefix(string[i:])
    n = len(temp)
    if n == 0:
      bit = convertToNineBits(convertToBinary(ord(string[i])))
      ret += bit
      i += 1
    else:
      bit = dic[temp]
      ret += bit
      i += n
  return ret



def decompress(string, dic):
  ret = ''
  for i in range(0, len(string) - 1, 9):
    if int(string[i : i + 9], 2) > 255:
      ret += dic[string[i : i + 9]]
    else:
      ret += chr(int(string[i : i + 9], 2))
  return ret

**Tests sur la base d'apprentissage du dictionnaire**

In [ ]:
# First dictionnary not based on a book

compressed = compress(text, tree, dic)

decompressed = decompress(compressed, reverted_dic)


print(f'Le fichier avant décompression faisait {len(text) * 8} octets et celui compressé fait {len(compressed)} octets')
print(f'Le taux de compression est donc de {len(text) * 8 / len(compressed)}')

Le fichier avant décompression faisait 5445800 octets et celui compressé fait 5363739 octets
Le taux de compression est donc de 1.0152992157150078


In [ ]:
# Second dictionnary (made with only strings of 3 char)

compressed2 = compress(text, tree2, dic2)

decompressed2 = decompress(compressed2, reverted_dic2)


print(f'Le fichier avant décompression faisait {len(text) * 8} octets et celui compressé fait {len(compressed2)} octets')
print(f'Le taux de compression est donc de {len(text) * 8 / len(compressed2)}')

Le fichier avant décompression faisait 5445800 octets et celui compressé fait 3425643 octets
Le taux de compression est donc de 1.5897161496396444


In [ ]:
# Third dictionnary (made with strings of 2-3 char)

compressed3 = compress(text, tree3, dic3)

decompressed3 = decompress(compressed3, reverted_dic3)


print(f'Le fichier avant décompression faisait {len(text) * 8} octets et celui compressé fait {len(compressed3)} octets')
print(f'Le taux de compression est donc de {len(text) * 8 / len(compressed3)}')

Le fichier avant décompression faisait 5445800 octets et celui compressé fait 3055878 octets
Le taux de compression est donc de 1.7820737607980424


In [16]:
# Fourth dictionnary (made with only strings of 2-9 char)

compressed4 = compress(text, tree4, dic4)

decompressed4 = decompress(compressed4, reverted_dic4)


print(f'Le fichier avant décompression faisait {len(text) * 8} octets et celui compressé fait {len(compressed4)} octets')
print(f'Le taux de compression est donc de {len(text) * 8 / len(compressed4)}')

Le fichier avant décompression faisait 5445800 octets et celui compressé fait 3034512 octets
Le taux de compression est donc de 1.7946213427397881


In [17]:
# Fifth dictionnary (made with only strings of 2-49 char)

compressed5 = compress(text, tree5, dic5)

decompressed5 = decompress(compressed5, reverted_dic5)


print(f'Le fichier avant décompression faisait {len(text) * 8} octets et celui compressé fait {len(compressed5)} octets')
print(f'Le taux de compression est donc de {len(text) * 8 / len(compressed5)}')

Le fichier avant décompression faisait 5445800 octets et celui compressé fait 3034512 octets
Le taux de compression est donc de 1.7946213427397881


**Test du dictionnaire sur un autre livre, ici le premier tome du livre "Les misérables"**

In [25]:
import textract
text2 = textract.process("dataFrenchWords/hugo_les_miserables_fantine_source.doc")
text2 = text2.decode("utf-8")
text2 = text2.replace('’', '\'').replace('–', '-').replace('…', '...').replace('‘', '\'').replace('€', 'euros').replace('[pic]', '')

In [26]:
compressed6 = compress(text2, tree5, dic5)

decompressed6 = decompress(compressed6, reverted_dic5)


print(f'Le fichier avant décompression faisait {len(text2) * 8} octets et celui compressé fait {len(compressed6)} octets')
print(f'Le taux de compression est donc de {len(text2) * 8 / len(compressed6)}')

Le fichier avant décompression faisait 6131592 octets et celui compressé fait 3502737 octets
Le taux de compression est donc de 1.7505145262119308
